In [1]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as funcs

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1651721918736_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("BigData").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data = spark.read.option("header",True).csv("s3n://big-data-bia678/final_merged_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
data.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ISIN: string, MarketSegment: string, UnderlyingSymbol: string, UnderlyingISIN: string, Currency: string, SecurityType: string, MaturityDate: string, StrikePrice: string, PutOrCall: string, MLEG: string, ContractGenerationNumber: string, SecurityID: string, Date: string, Time: string, StartPrice: string, MaxPrice: string, MinPrice: string, EndPrice: string, NumberOfContracts: string, NumberOfTrades: string]

In [5]:
data.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ISIN: string, MarketSegment: string, UnderlyingSymbol: string, UnderlyingISIN: string, Currency: string, SecurityType: string, MaturityDate: string, StrikePrice: string, PutOrCall: string, MLEG: string, ContractGenerationNumber: string, SecurityID: string, Date: string, Time: string, StartPrice: string, MaxPrice: string, MinPrice: string, EndPrice: string, NumberOfContracts: string, NumberOfTrades: string]

In [6]:
data = data.dropDuplicates()
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+----------------+--------------+--------+------------+------------+-----------+---------+--------------------+------------------------+----------+----------+-----+----------+--------+--------+--------+-----------------+--------------+
|        ISIN|MarketSegment|UnderlyingSymbol|UnderlyingISIN|Currency|SecurityType|MaturityDate|StrikePrice|PutOrCall|                MLEG|ContractGenerationNumber|SecurityID|      Date| Time|StartPrice|MaxPrice|MinPrice|EndPrice|NumberOfContracts|NumberOfTrades|
+------------+-------------+----------------+--------------+--------+------------+------------+-----------+---------+--------------------+------------------------+----------+----------+-----+----------+--------+--------+--------+-----------------+--------------+
|DE000C58X4H8|         FDXM|             DAX|  DE0008469008|     EUR|         FUT|  20211217.0|       null|     null| FDXM SI 20211217 CS|                    null| 6209447.0|2021-11-09|10:20|   16077.0| 16078.0|

In [7]:
from pyspark.sql.types import DoubleType
data = data.withColumn("StrikePrice", data["StrikePrice"].cast(DoubleType()))
data= data.withColumn("StartPrice", data["StartPrice"].cast(DoubleType()))
data = data.withColumn("MaxPrice", data["MaxPrice"].cast(DoubleType()))
data = data.withColumn("MinPrice", data["MinPrice"].cast(DoubleType()))
data = data.withColumn("EndPrice", data["EndPrice"].cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
numeric_columns = list()
categorical_column = list()
for col_ in data.columns:
    if data.select(col_).dtypes[0][1] != "string":
        numeric_columns.append(col_)
    else:
        categorical_column.append(col_)
        
print("Numeric columns",numeric_columns)
print("categorical columns",categorical_column)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Numeric columns ['StrikePrice', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice']
categorical columns ['ISIN', 'MarketSegment', 'UnderlyingSymbol', 'UnderlyingISIN', 'Currency', 'SecurityType', 'MaturityDate', 'PutOrCall', 'MLEG', 'ContractGenerationNumber', 'SecurityID', 'Date', 'Time', 'NumberOfContracts', 'NumberOfTrades']

In [9]:
from pyspark.sql.functions import *
nan_rows = data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
data = data.drop("UnderlyingISIN", "UnderlyingSymbol", "SecurityID")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
data = data.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
import pyspark.sql.functions as func

date_from = "2021-01-01" 
date_to = "2022-01-01"
#data = data.select(func.to_date(data["Date"]).alias("time"))
sf = data.filter(data.Date > date_from).filter(data.Date < date_to)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
sf.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ISIN='DE000C1W2M60', MarketSegment='ODX2', Currency='EUR', SecurityType='OPT', MaturityDate='20211112.0', StrikePrice=16100.0, PutOrCall='Put', MLEG='ODX2 SI 20211112 CS EU P 16100 0', ContractGenerationNumber='1.0', Date='2021-11-09', Time='10:33', StartPrice=76.5, MaxPrice=76.5, MinPrice=76.5, EndPrice=76.5, NumberOfContracts='1.0', NumberOfTrades='1.0')]

In [14]:
#Groupby put or call
sf.groupBy('PutOrCall').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+
|PutOrCall|  count|
+---------+-------+
|      Put|2876277|
|     Call|2256072|
+---------+-------+

In [16]:
#Groupby currency
sf.groupBy('Currency').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+
|Currency|  count|
+--------+-------+
|     CHF| 393867|
|     GBX|   1634|
|     USD|    823|
|     EUR|2824787|
|     KRW|1911232|
|     GBP|      6|
+--------+-------+

In [59]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="SecurityType", outputCol="sec")
indexed = indexer.fit(sf).transform(sf)
indexed.head(5)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ISIN='DE000C1W2M60', MarketSegment='ODX2', Currency='EUR', SecurityType='OPT', MaturityDate='20211112.0', StrikePrice=16100.0, PutOrCall='Put', MLEG='ODX2 SI 20211112 CS EU P 16100 0', ContractGenerationNumber='1.0', Date='2021-11-09', Time='10:33', StartPrice=76.5, MaxPrice=76.5, MinPrice=76.5, EndPrice=76.5, NumberOfContracts='1.0', NumberOfTrades='1.0', sec=0.0), Row(ISIN='DE000C0PLE53', MarketSegment='OESX', Currency='EUR', SecurityType='OPT', MaturityDate='20211217.0', StrikePrice=4200.0, PutOrCall='Put', MLEG='OESX SI 20211217 CS EU P 4200 0', ContractGenerationNumber='1.0', Date='2021-11-09', Time='11:26', StartPrice=36.8, MaxPrice=36.8, MinPrice=36.8, EndPrice=36.8, NumberOfContracts='201.0', NumberOfTrades='2.0', sec=0.0), Row(ISIN='DE000C1WYZT5', MarketSegment='SZG', Currency='EUR', SecurityType='OPT', MaturityDate='20220121.0', StrikePrice=34.0, PutOrCall='Call', MLEG='SZG SI 20220121 PS AM C 34.00 0', ContractGenerationNumber='4.0', Date='2021-11-09', Time='13:14', Sta

In [70]:
#multiple column featurizer.
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(sf) for column in list(set(sf.columns)-set(['date'])) ]

pipeline = Pipeline(stages=indexers)
indexed = pipeline.fit(sf).transform(sf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
indexed.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ISIN='DE000C1W2M60', MarketSegment='ODX2', Currency='EUR', SecurityType='OPT', MaturityDate='20211112.0', StrikePrice=16100.0, PutOrCall='Put', MLEG='ODX2 SI 20211112 CS EU P 16100 0', ContractGenerationNumber='1.0', Date='2021-11-09', Time='10:33', StartPrice=76.5, MaxPrice=76.5, MinPrice=76.5, EndPrice=76.5, NumberOfContracts='1.0', NumberOfTrades='1.0', PutOrCall_index=0.0, StrikePrice_index=182.0, ContractGenerationNumber_index=0.0, StartPrice_index=965.0, Currency_index=0.0, EndPrice_index=969.0, NumberOfContracts_index=5.0, ISIN_index=8026.0, Date_index=158.0, MLEG_index=7986.0, MarketSegment_index=19.0, MaxPrice_index=969.0, SecurityType_index=0.0, MaturityDate_index=63.0, NumberOfTrades_index=2.0, MinPrice_index=961.0, Time_index=201.0)]

In [72]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
sf.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ISIN', 'MarketSegment', 'Currency', 'SecurityType', 'MaturityDate', 'StrikePrice', 'PutOrCall', 'MLEG', 'ContractGenerationNumber', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'NumberOfContracts', 'NumberOfTrades']

In [119]:

assembler = VectorAssembler(
  inputCols=['SecurityType_index',
             #'MLEG_index',
             'StartPrice',
             'MaxPrice',
             'MinPrice',
             #'MarketSegment_index',
             'StrikePrice'
            ],
    outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
output = assembler.transform(indexed)
output.select("features", "EndPrice").show()
final_data = output.select("features", "EndPrice")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='EndPrice')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coefficients: [0.0,-0.7895038597924635,0.6970573469102191,1.092448337370349,1.1862658397500759e-06] Intercept: 0.0006751078477741135

In [126]:
test_results = lrModel.evaluate(test_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE: 0.07517837747196177
MSE: 0.0056517884393167675
R2: 0.9999998033353552

In [149]:
#Strike price prediction
assembler = VectorAssembler(
  inputCols=['SecurityType_index',
             'MLEG_index',
             'PutOrCall_index',
             'StartPrice',
             'MaxPrice',
             'MinPrice',
             'EndPrice',
             'MarketSegment_index'
            ],
    outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
output = assembler.transform(indexed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
output.select("features", "StrikePrice").show()
final_data = output.select("features", "StrikePrice")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|            features|StrikePrice|
+--------------------+-----------+
|[0.0,7986.0,0.0,7...|    16100.0|
|[0.0,374.0,0.0,36...|     4200.0|
|[0.0,113921.0,1.0...|       34.0|
|[0.0,9847.0,0.0,0...|      170.5|
|[0.0,47.0,1.0,0.2...|      397.5|
|[0.0,11.0,0.0,0.6...|      385.0|
|[0.0,47.0,1.0,0.3...|      397.5|
|[0.0,17912.0,0.0,...|     3975.0|
|[0.0,87023.0,0.0,...|      100.0|
|[0.0,214.0,1.0,2....|      390.0|
|[0.0,66.0,0.0,0.0...|      375.0|
|[0.0,25766.0,0.0,...|        4.0|
|[0.0,25263.0,0.0,...|       12.6|
|[0.0,17.0,1.0,0.0...|      400.0|
|[0.0,67.0,1.0,0.4...|      395.0|
|[0.0,48.0,0.0,3.0...|      390.0|
|[0.0,167.0,1.0,0....|      392.5|
|[0.0,12481.0,0.0,...|       43.5|
|[0.0,4.0,1.0,1.46...|      402.5|
|[0.0,23022.0,0.0,...|      161.0|
+--------------------+-----------+
only showing top 20 rows

In [152]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='StrikePrice')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coefficients: [0.0,-0.006268962377046943,-14.769061909112818,234.00185753585043,1391.9325096706416,-1389.746396593621,-226.7172570459919,-6.520406974828376] Intercept: 2288.3953227901006

In [156]:
test_results = lrModel.evaluate(test_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE: 4115.592767868261
MSE: 16938103.830929536
R2: 0.1407630371203058